In [6]:
import numpy as np
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC

In [11]:
def load_image_files(container_path, dimension=(128,128)):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    
    # Get all classes (categories)
    categories = [fo.name for fo in folders]

    descr = "Your own dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            if file.suffix.lower() not in ['.jpg', '.jpeg', '.png', '.webp', '.gif']:
                print(f"Skipped non-image file: {file}")
                continue
            img = cv2.imread(str(file), cv2.IMREAD_COLOR)
            if img is None:
                print(f"Failed to read image: {file}")
                continue
            img_resized = cv2.resize(img, dimension, interpolation=cv2.INTER_AREA)
            if img_resized.size == 0:
                print(f"Empty image: {file}")
                continue
            flat_data.append(img_resized.flatten())
            images.append(img_resized)
            target.append(i)
    
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(
        data=flat_data,
        target=target,
        target_names=categories,
        images=images,
        DESCR=descr
    )


In [12]:
image_dataset = load_image_files("/home/akill_sud/Documents/projects/soil project/Soil_Data_V3/Trains")
image_dataset_test = load_image_files("/home/akill_sud/Documents/projects/soil project/Soil_Data_V3/Tests/")

Skipped non-image file: /home/akill_sud/Documents/projects/soil project/Soil_Data_V3/Trains/Silt/desktop.ini
Skipped non-image file: /home/akill_sud/Documents/projects/soil project/Soil_Data_V3/Trains/Mary/desktop.ini
Skipped non-image file: /home/akill_sud/Documents/projects/soil project/Soil_Data_V3/Trains/Sand/desktop.ini
Skipped non-image file: /home/akill_sud/Documents/projects/soil project/Soil_Data_V3/Tests/Chalky/desktop.ini


In [13]:
print(image_dataset.target_names)

['Silt', 'Red soil', 'Mary', 'Chalky', 'Clay soil', 'Sand', 'Alluvial soil', 'Black']


In [14]:
X_train= image_dataset.images
X_test= image_dataset_test.images
y_train= image_dataset.target
y_test=image_dataset_test.target

print(X_train.shape)
print(X_test.shape)

(3670, 128, 128, 3)
(1584, 128, 128, 3)


In [15]:
#Trial
from keras.models import Sequential

from keras.layers import Conv2D, AveragePooling2D,Activation,Dropout, Flatten, BatchNormalization, Dense

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128,128,3))) # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))    

model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.2))


model.add(Conv2D(64,(3, 3))) 
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))   
model.add(AveragePooling2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.2))
          
model.add(Conv2D(128,(3, 3))) 
model.add(Conv2D(128,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))  
model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())                                 # Flatten final 4x4x64 output matrix into a 1024-length vector
model.add(Dropout(0.2)) 


# Fully Connected Layer 5
model.add(Dense(512))                                # 512 FCN nodes
model.add(BatchNormalization())                      # normalization
model.add(Activation('relu'))                        # activation
model.add(Dropout(0.2))

model.add(Dense(8))                                 # final 10 FCN nodes
model.add(Activation('softmax')) 

/home/akill_sud/anaconda3/envs/gpu_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-10 23:39:44.410843: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 63, 63, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 63, 63, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 30, 30, 64)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 26, 26, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 26, 26, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 13, 13, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    11,076,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 512)            │         2,04

 Total params: 11,324,360 (43.20 MB)

 Trainable params: 11,322,696 (43.19 MB)

 Non-trainable params: 1,664 (6.50 KB)

In [17]:
# we'll use the same optimizer

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss",
                                        mode="min", patience=10,
                                        restore_best_weights=True)

In [18]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [20]:
cvscores = []
for train, test in kfold.split(X_train, y_train):
    history=model.fit(X_train, y_train, batch_size=8, epochs=10, validation_data=(X_test,y_test),
          verbose=1, callbacks=[earlystopping])
    # evaluate the model
    scores = model.evaluate(X_train[test], y_train[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 67s 146ms/step - accuracy: 0.3793 - loss: 1.6888 - val_accuracy: 0.2891 - val_loss: 2.4379
Epoch 2/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 66s 144ms/step - accuracy: 0.4617 - loss: 1.4701 - val_accuracy: 0.5006 - val_loss: 1.4046
Epoch 3/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 67s 145ms/step - accuracy: 0.5435 - loss: 1.3223 - val_accuracy: 0.5297 - val_loss: 1.3449
Epoch 4/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 67s 147ms/step - accuracy: 0.6433 - loss: 1.0403 - val_accuracy: 0.6850 - val_loss: 1.0167
Epoch 5/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 69s 149ms/step - accuracy: 0.7594 - loss: 0.7385 - val_accuracy: 0.7292 - val_loss: 0.8819
Epoch 6/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 68s 149ms/step - accuracy: 0.8412 - loss: 0.5104 - val_accuracy: 0.7904 - val_loss: 0.7697
Epoch 7/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 69s 150ms/step - accuracy: 0.8779 - loss: 0.3639 - val_accuracy: 0.7715 - val_loss: 0.8514
Epoch 8/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 67s 147ms/step - accuracy: 0.8981 - loss: 0

KeyboardInterrupt: 

In [21]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step


In [22]:
print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.90      0.74      0.81       168
           1       0.94      0.98      0.96       222
           2       0.87      0.92      0.90       172
           3       0.74      0.79      0.77       166
           4       0.96      0.72      0.83       160
           5       0.79      0.82      0.80       170
           6       0.87      0.94      0.91       316
           7       0.90      0.94      0.92       210

    accuracy                           0.87      1584
   macro avg       0.87      0.86      0.86      1584
weighted avg       0.88      0.87      0.87      1584



In [23]:
model.save('soil_detection_model.h5')